# MITSUI&CO. Commodity Prediction Challenge

- Implementing a baseline XGBoost model to predict commodity return differences.
- The goal is to maximize the rank correlation Sharpe ratio.

In [2]:
# !pip install xgboost pandas numpy scikit-learn matplotlib seaborn polars optuna

In [2]:
# Imports
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import polars as pl
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'polars'

In [ ]:
# Set device
try:
    xgb_gpu_test = xgb.XGBRegressor(tree_method='gpu_hist')
    xgb_gpu_test.fit(np.array([[1,2], [3,4]]), np.array([1,2]))
    GPU_AVAILABLE = True
    print("GPU is available for XGBoost training")
except:
    GPU_AVAILABLE = False
    print("GPU not available, using CPU for XGBoost")

### Evaluation Metric Implementation

In [ ]:
# Define function to compute evaluation metric
SOLUTION_NULL_FILLER = -999999

def rank_correlation_sharpe_ratio(merged_df: pd.DataFrame) -> float:
    prediction_cols = [col for col in merged_df.columns if col.startswith('prediction_')]
    target_cols = [col for col in merged_df.columns if col.startswith('target_')]

    def _compute_rank_correlation(row):
        non_null_targets = [col for col in target_cols if not pd.isnull(row[col])]
        matching_predictions = [col for col in prediction_cols if col.replace('prediction', 'target') in non_null_targets]
        if not non_null_targets:
            return 0.0  # Return 0 instead of raising error for robustness
        if row[non_null_targets].std(ddof=0) == 0 or row[matching_predictions].std(ddof=0) == 0:
            return 0.0  # Return 0 instead of raising error for robustness
        return np.corrcoef(row[matching_predictions].rank(method='average'), 
                          row[non_null_targets].rank(method='average'))[0, 1]

    daily_rank_corrs = merged_df.apply(_compute_rank_correlation, axis=1)
    
    # Handle cases where all correlations are 0
    if daily_rank_corrs.std(ddof=0) == 0:
        return daily_rank_corrs.mean()
    
    sharpe_ratio = daily_rank_corrs.mean() / daily_rank_corrs.std(ddof=0)
    return float(sharpe_ratio)

def competition_score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    if row_id_column_name in solution.columns:
        del solution[row_id_column_name]
    if row_id_column_name in submission.columns:
        del submission[row_id_column_name]
    
    # Ensure both dataframes have the same columns
    common_cols = list(set(solution.columns) & set(submission.columns))
    solution = solution[common_cols]
    submission = submission[common_cols]
    
    submission = submission.rename(columns={col: col.replace('target_', 'prediction_') for col in submission.columns})
    solution = solution.replace(SOLUTION_NULL_FILLER, None)
    
    # Drop rows where all targets are null
    valid_rows = ~solution.isnull().all(axis=1)
    solution = solution[valid_rows]
    submission = submission[valid_rows]
    
    if len(solution) == 0:
        return 0.0
    
    return rank_correlation_sharpe_ratio(pd.concat([solution, submission], axis='columns'))

# Custom scorer for sklearn
def sharpe_ratio_scorer(y_true, y_pred):
    # Create a DataFrame with target columns
    solution = pd.DataFrame(y_true)
    solution.columns = [f'target_{i}' for i in range(y_true.shape[1])]
    
    # Create submission with same structure
    submission = pd.DataFrame(y_pred)
    submission.columns = [f'target_{i}' for i in range(y_pred.shape[1])]
    
    # Add row_id for compatibility
    solution['row_id'] = range(len(solution))
    submission['row_id'] = range(len(submission))
    
    return competition_score(solution, submission, 'row_id')

# Make it a scorer object
competition_scorer = make_scorer(sharpe_ratio_scorer, greater_is_better=True)

print("Evaluation metric functions loaded")

### Data Loading and Initial Inspection

In [ ]:
# Load datasets 
def load_data():
    data_path = './' 
    
    # Load training data
    train = pd.read_csv(f'{data_path}train.csv')
    train_labels = pd.read_csv(f'{data_path}train_labels.csv')
    target_pairs = pd.read_csv(f'{data_path}target_pairs.csv')
    
    # Load test data
    test = pd.read_csv(f'{data_path}test.csv')
    return train, train_labels, target_pairs, test

In [ ]:
# Inspect respective shapes of loaded datasets
train, train_labels, target_pairs, test = load_data()
print(f"\n Training data shape: {train.shape}")
print(f" Training labels shape: {train_labels.shape}")
print(f" Target pairs info: {target_pairs.shape} rows")
print(f"\n Test data shape: {test.shape}")

In [ ]:
train.head(3)

In [ ]:
train_labels.head(3)

In [ ]:
target_pairs.head()

### Data Cleaning and Preprocessing

In [ ]:
# Define function to clean the train, train_labels, and target_pairs datasets
def clean_data(train, train_labels, target_pairs):
    # Check for missing values
    print("\n Missing values in train data:")
    missing_train = train.isnull().sum()
    print(missing_train[missing_train > 0])
    
    print("\n Missing values in train labels:")
    missing_labels = train_labels.isnull().sum()
    print(missing_labels[missing_labels > 0])
    
    # Handle missing values in train data
    print("\n Handling missing values...")
    train_clean = train.copy()
    
    # Fill date-related features
    if 'date' in train_clean.columns:
        train_clean['date'] = pd.to_datetime(train_clean['date'])
        train_clean = train_clean.sort_values('date')
    
    # Forward fill for time series continuity
    train_clean = train_clean.fillna(method='ffill')
    # Then backfill any remaining at the beginning
    train_clean = train_clean.fillna(method='bfill')
    
    # Check for duplicates
    duplicates = train_clean.duplicated().sum()
    print(f"\n Found {duplicates} duplicate rows - removing...")
    train_clean = train_clean.drop_duplicates()
    
    # Handle outliers using IQR method
    print("\n Handling outliers...")
    numeric_cols = train_clean.select_dtypes(include=[np.number]).columns
    train_clean_no_outliers = train_clean.copy()
    
    for col in numeric_cols:
        Q1 = train_clean[col].quantile(0.01)
        Q3 = train_clean[col].quantile(0.99)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers
        outliers = ((train_clean[col] < lower_bound) | (train_clean[col] > upper_bound)).sum()
        if outliers > 0:
            print(f"  Column '{col}': {outliers} outliers detected")
            
            # Cap outliers instead of removing
            train_clean_no_outliers[col] = train_clean[col].clip(lower_bound, upper_bound)
    
    # Check label data
    print("\n Label data statistics:")
    print(train_labels.describe().T.head())
    
    # Verify target pairs
    print("\n Target pairs info:")
    print(f"Unique asset combinations: {len(target_pairs[['asset1', 'asset2']].drop_duplicates())}")
    print(f"Calculation types: {target_pairs['calculation'].value_counts().to_dict()}")
    
    # Align train and label data
    if 'date' in train_clean_no_outliers.columns and len(train_clean_no_outliers) == len(train_labels):
        print("\n Aligning train and label data by date")
        
        # Set the date feature as the indexn
        train_clean_no_outliers = train_clean_no_outliers.reset_index(drop=True)
        train_labels = train_labels.reset_index(drop=True)
    
    print("\n Data cleaning completed!")
    return train_clean_no_outliers, train_labels, target_pairs

In [ ]:
# Clean the train, train_labels, and target_pairs datasets
train_clean, train_labels, target_pairs = clean_data(train, train_labels, target_pairs)

### Exploratory Data Analysis 

In [ ]:
# Define function to perform EDA on train, train_labels, and target_pairs datasets
def perform_eda(train, train_labels, target_pairs):
    # Basic dataset information
    print(f"Training features: {train.shape[0]} samples, {train.shape[1]} features")
    print(f"Training labels: {train_labels.shape[0]} samples, {train_labels.shape[1]} targets")
    
    # Feature distributions
    print("\n Numeric Features Distributions")
    numeric_cols = train.select_dtypes(include=[np.number]).columns[:5]
    plt.figure(figsize=(15, 10))
    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(2, 3, i)
        sns.histplot(train[col], kde=True)
        plt.title(f'Distribution of {col}')
    plt.tight_layout()
    plt.show()
    
    # Perform correlation analysis
    print("\n Compute feature correlations")
    # Sample a subset to plot a correlation matrix
    sample_size = min(1000, len(train))
    sample_train = train.sample(sample_size, random_state=42)
    
    # Select top 10 numeric features 
    numeric_cols = sample_train.select_dtypes(include=[np.number]).columns[:10]
    corr_matrix = sample_train[numeric_cols].corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
    plt.title('Feature Correlation Matrix (First 10 Features)')
    plt.tight_layout()
    plt.show()
    
    # Analyze Target variables
    print("\n Analyze target variables")
    # Sample targets for visualization
    sample_targets = train_labels.sample(sample_size, random_state=42)
    
    # Plot distribution of first 5 targets
    plt.figure(figsize=(15, 10))
    for i in range(5):
        if i < sample_targets.shape[1]: 
            plt.subplot(2, 3, i+1)
            sns.histplot(sample_targets[f'target_{i}'], kde=True)
            plt.title(f'Distribution of target_{i}')
    plt.tight_layout()
    plt.show()
    
    # Perform Time series analysis 
    if 'date' in train.columns:
        print("\n Performing time series analysis...")
        train_with_date = train.copy()
        train_with_date['date'] = pd.to_datetime(train_with_date['date'])
        train_with_date = train_with_date.set_index('date')
        
        plt.figure(figsize=(14, 10))
        for i, col in enumerate(numeric_cols[:3], 1):
            plt.subplot(3, 1, i)
            plt.plot(train_with_date.index, train_with_date[col])
            plt.title(f'Time Series: {col}')
            plt.tight_layout()
        plt.show()
    
    # \perform Target Pair Analysis
    print("\n Analyze target pairs")
    plt.figure(figsize=(10, 6))
    target_pairs['asset1'].value_counts().plot(kind='bar')
    plt.title('Distribution of Asset 1 in Target Pairs')
    plt.tight_layout()
    plt.show()
    
    # Compute and display the competition metric on a random prediction
    print("\n Calculate baseline competition metric for random predictions)")
    random_preds = np.random.randn(len(train_labels), train_labels.shape[1])
    solution_df = train_labels.copy()
    submission_df = pd.DataFrame(random_preds, columns=train_labels.columns)
    
    # Add row_id for the metric function
    solution_df['row_id'] = range(len(solution_df))
    submission_df['row_id'] = range(len(submission_df))
    baseline_score = competition_score(solution_df, submission_df, 'row_id')
    print(f"Baseline random prediction score (Sharpe ratio): {baseline_score:.4f}")



In [ ]:
# Perform EDA on cleaned train, train_labels, and target_pairs datasets
perform_eda(train_clean, train_labels, target_pairs)

### Feature Engineering

In [ ]:
# Define function to perform feature engineering
def feature_engineering(train, train_labels):
    train_eng = train.copy()
    
    # Date-based features (if date column exists)
    if 'date' in train_eng.columns:
        print("Create date-based features")
        train_eng['date'] = pd.to_datetime(train_eng['date'])
        train_eng['day_of_week'] = train_eng['date'].dt.dayofweek
        train_eng['month'] = train_eng['date'].dt.month
        train_eng['quarter'] = train_eng['date'].dt.quarter
        train_eng['is_weekend'] = train_eng['day_of_week'].isin([5, 6]).astype(int)
        
        # Drop the original date column
        train_eng = train_eng.drop('date', axis=1)
    
    # Technical indicators for numeric features
    print(" Create technical indicators")
    numeric_cols = train_eng.select_dtypes(include=[np.number]).columns
    
    # Moving averages for key features
    for col in numeric_cols[:10]:  # Just a subset to avoid too many features
        train_eng[f'{col}_ma_5'] = train_eng[col].rolling(window=5, min_periods=1).mean()
        train_eng[f'{col}_ma_20'] = train_eng[col].rolling(window=20, min_periods=1).mean()
        train_eng[f'{col}_vol_20'] = train_eng[col].rolling(window=20, min_periods=1).std()
        train_eng[f'{col}_mom_10'] = train_eng[col].diff(10)
    
    # Cross-feature interactions
    print(" Create cross-feature interactions")
    # Only for the most important features (to avoid explosion of features)
    for i in range(min(5, len(numeric_cols))):
        for j in range(i+1, min(8, len(numeric_cols))):
            col1, col2 = numeric_cols[i], numeric_cols[j]
            train_eng[f'{col1}_x_{col2}'] = train_eng[col1] * train_eng[col2]
            train_eng[f'{col1}_{col2}_ratio'] = train_eng[col1] / (train_eng[col2] + 1e-8)  # Avoid division by zero
    
    # Rank normalization for all numeric features
    print(" Apply rank normalization to numeric features")
    for col in numeric_cols:
        train_eng[col] = train_eng[col].rank(pct=True)
    
    # Handle any remaining NaNs after feature engineering
    train_eng = train_eng.fillna(0.5)  # Impute NaNs with median rank
    
    print(f" New shape: {train_eng.shape}")
    return train_eng

In [ ]:
# Perform perform feature engineering train_clean and train_labels
train_eng = feature_engineering(train_clean, train_labels)

### Data Prepration

In [ ]:
# Define function to prepare data for modelling based on a time-based split
def prepare_model_data(train_eng, train_labels):  
    # Convert to numpy arrays for XGBoost
    X = train_eng.select_dtypes(include=[np.number]).values
    y = train_labels.values
    print(f"Final feature matrix shape: {X.shape}")
    print(f"Target matrix shape: {y.shape}")
    
    # Perform Train-Test split
    split_idx = int(0.8 * len(X))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Validation set: {X_val.shape[0]} samples")
    
    return X_train, X_val, y_train, y_val

In [ ]:
# Prepare data for modelling
X_train, X_val, y_train, y_val = prepare_model_data(train_eng, train_labels)

### Modelling

In [ ]:
# Train a baseline XGBoost model
def create_xgboost_model(gpu_available=False):   
    # Define Base parameters
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': 0.1,
        'max_depth': 6,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'tree_method': 'gpu_hist' if gpu_available else 'auto',
        'random_state': 42,
        'n_estimators': 100
    }
    
    print(f" XGBoost parameters: {params}")
    
    # Create the model
    model = xgb.XGBRegressor(**params)
    return model, params

# Create and train the model
def train_baseline_model(model, X_train, y_train, X_val, y_val):  
    # Train the model
    start_time = time.time()
    
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=20,
        verbose=10
    )
    
    train_time = time.time() - start_time
    print(f" Baseline Vanilla XGBoost model training time: {train_time:.2f} seconds")
    
    # Make predictions
    val_preds = model.predict(X_val)
    
    # Calculate competition metric
    val_solution = pd.DataFrame(y_val, columns=[f'target_{i}' for i in range(y_val.shape[1])])
    val_submission = pd.DataFrame(val_preds, columns=[f'target_{i}' for i in range(y_val.shape[1])])
    
    # Add row_id for metric calculation
    val_solution['row_id'] = range(len(val_solution))
    val_submission['row_id'] = range(len(val_submission))
    
    score_val = competition_score(val_solution, val_submission, 'row_id')
    print(f" Baseline Vanilla XGBoost model validation score (Sharpe ratio): {score_val:.4f}")
    
    # Compute and visualize feature importance (top-20 features)
    if hasattr(model, 'feature_importances_'):
        plt.figure(figsize=(12, 8))
        feat_importances = pd.Series(model.feature_importances_, index=train_eng.columns)
        feat_importances.nlargest(20).plot(kind='barh')
        plt.title('Top 20 Feature Importances')
        plt.tight_layout()
        plt.show()
    
    return model, val_preds, score_val



In [ ]:
# Create baseline vanilla XGBoost model
model, base_params = create_xgboost_model(GPU_AVAILABLE)

In [ ]:
# Train baseline vanilla XGBoost model
model, val_preds, baseline_score = train_baseline_model(model, X_train, y_train, X_val, y_val)

#### Hyperparameter Optimization
- Define function to optimize the baseline vanilla XGBoost model using RandomSearchCV.

In [ ]:
def optimize_model(X_train, y_train, X_val, y_val, gpu_available=False):  
    # Define parameter grid
    param_dist = {
        'n_estimators': [100, 200, 300, 500],
        'max_depth': [3, 5, 7, 9, 11],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2, 0.3, 0.4],
        'min_child_weight': [1, 3, 5, 7],
        'reg_alpha': [0, 0.1, 0.5, 1.0],
        'reg_lambda': [0.5, 1.0, 1.5, 2.0]
    }
    
    # Set device
    if gpu_available:
        param_dist['tree_method'] = ['gpu_hist']
    else:
        param_dist['tree_method'] = ['auto']
    
    print("Starting RandomizedSearchCV...")
    print(f"Parameter grid size: {np.prod([len(v) for v in param_dist.values()])} combinations")
    print(f"Sampling 20 parameter combinations")
    
    # Perform a Time series split for validation
    tscv = TimeSeriesSplit(n_splits=3)
    
    # Create base model
    base_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        random_state=42
    )
    
    # RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=base_model,
        param_distributions=param_dist,
        n_iter=20,  
        scoring=competition_scorer,
        cv=tscv,
        verbose=2,
        random_state=42,
        n_jobs=-1
    )
    
    # Fit RandomizedSearchCV
    start_time = time.time()
    random_search.fit(X_train, y_train)
    search_time = time.time() - start_time
    
    print(f"\nRandomizedSearchCV optimization time: {search_time:.2f} seconds")
    print(f"Best parameters: {random_search.best_params_}")
    print(f"Best cross-validated score: {random_search.best_score_:.4f}")
    
    # Train the best param model on training set
    best_params = random_search.best_params_
    best_model = xgb.XGBRegressor(**best_params, objective='reg:squarederror')
    
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=20,
        verbose=10
    )
    
    # Evaluate optimized XGBoost model on validation set
    val_preds = best_model.predict(X_val)
    val_solution = pd.DataFrame(y_val, columns=[f'target_{i}' for i in range(y_val.shape[1])])
    val_submission = pd.DataFrame(val_preds, columns=[f'target_{i}' for i in range(y_val.shape[1])])
    val_solution['row_id'] = range(len(val_solution))
    val_submission['row_id'] = range(len(val_submission))
    
    optimized_score = competition_score(val_solution, val_submission, 'row_id')
    print(f" Optimized XGBoost model validation score (Sharpe ratio): {optimized_score:.4f}")
    print(f" Improvement over baseline: {(optimized_score - baseline_score) / abs(baseline_score) * 100:.2f}%")
    
    return best_model, best_params, optimized_score

### XGBoost Model Evaluation and Inference Setup

In [ ]:
# Final model evaluation
print("\n RandomSearchCV optimized XGBoost Model Evaluation")
print(f"Baseline model score: {baseline_score:.4f}")
print(f"Optimized model score: {optimized_score:.4f}")

# Save optimized model
model_filename = 'commodity_prediction_model.pkl'
joblib.dump(best_model, model_filename)
print(f"\n Model saved to {model_filename}")

In [ ]:
import os
import time
import joblib
import numpy as np
import pandas as pd
import polars as pl
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

import kaggle_evaluation.mitsui_inference_server

NUM_TARGET_COLUMNS = 424

model = None

def predict(
    test: pl.DataFrame,
    label_lags_1_batch: pl.DataFrame,
    label_lags_2_batch: pl.DataFrame,
    label_lags_3_batch: pl.DataFrame,
    label_lags_4_batch: pl.DataFrame,
) -> pd.DataFrame:

    global model
    if model is None:
        start_time = time.time()
        print(" Loading model for first prediction...")
        
        model_path = 'commodity_prediction_model.pkl'

    # Convert Polars DataFrame to Pandas for processing
    test_pd = test.to_pandas()
    
    # Perform feature engineering on test set
    # Engineer date-based features
    if 'date' in test_pd.columns:
        test_pd['date'] = pd.to_datetime(test_pd['date'])
        test_pd['day_of_week'] = test_pd['date'].dt.dayofweek
        test_pd['month'] = test_pd['date'].dt.month
        test_pd['quarter'] = test_pd['date'].dt.quarter
        test_pd['is_weekend'] = test_pd['day_of_week'].isin([5, 6]).astype(int)
        test_pd = test_pd.drop('date', axis=1)
    
    # Engineer rank normalization for numeric features
    numeric_cols = test_pd.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        # Rank and convert to percentile
        test_pd[col] = test_pd[col].rank(pct=True).fillna(0.5)
    
    # Handle any remaining NaNs
    test_pd = test_pd.fillna(0.5)
    
    # Make prediction (one row per batch in the API)
    try:
        prediction = model.predict(test_pd)[0]
    except Exception as e:
        print(f"⚠️ Prediction error: {str(e)} - using zeros")
        prediction = np.zeros(NUM_TARGET_COLUMNS)
    
    # Format output as Pandas DataFrame 
    predictions = pd.DataFrame({
        f'target_{i}': [float(prediction[i])] for i in range(NUM_TARGET_COLUMNS)
    })
    
    # Verify output 
    assert isinstance(predictions, pd.DataFrame), "Output must be a Pandas DataFrame"
    assert len(predictions) == 1, "Output must contain exactly one row"
    assert predictions.shape[1] == NUM_TARGET_COLUMNS, f"Output must have {NUM_TARGET_COLUMNS} columns"
    
    return predictions

### Submission

In [ ]:
import kaggle_evaluation.mitsui_inference_server

# Instantiate inference server
inference_server = kaggle_evaluation.mitsui_inference_server.MitsuiInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(('/kaggle/input/mitsui-commodity-prediction-challenge/',))